## RDP - Stone Soup Experiments
[Documentation](https://stonesoup.readthedocs.io/en/v1.4/auto_examples/readers/Custom_Pandas_Dataloader.html#sphx-glr-auto-examples-readers-custom-pandas-dataloader-py)

Continuing experiments - following tutorial #6: Data association - multi-target tracking tutorial.

For this tutorial, I need to extend the test data set from the previous notebook to add a second target. I'll look for a target of opportunity that is observed at the same time as the target considered so far. If possible, a few different cases would be interesting:
1. A straight flight that does not interesect the first flight path
1. A straight flight that DOES intersect the first flight path, but at a different time: 10610889
1. A non-linear flight that does not cross the first flight
1. A non-linear flight that DOES cross the first flight path

In [ ]:
import pandas as pd
import numpy as np
import csv
from datetime import datetime, timedelta
from importlib import reload  # Python 3.4+
from typing import Tuple
import itertools
from matplotlib import pyplot as plt
from math import ceil
from ordered_set import OrderedSet

import dateutil
from pymap3d import geodetic2enu

import sys
sys.path.append('C:/Users/ttrinter/git_repo/cspeed/data_common')
sys.path.append('../../..')
import data_functions as dfunc
import visualizations as v
from ttt_ss_funcs import generate_timestamps, ADSBTruthReader, CSVReaderXY, CSVReaderPolar, plot_all, CSVClutterReaderXY, group_plots


from stonesoup.reader import DetectionReader, GroundTruthReader
from stonesoup.base import Property
from stonesoup.types.detection import Detection, Clutter
from stonesoup.plotter import AnimatedPlotterly, Plotter

from stonesoup.base import Property
from stonesoup.buffered_generator import BufferedGenerator
from stonesoup.functions import cart2sphere, sphere2cart
from stonesoup.models.measurement.linear import LinearGaussian
from stonesoup.models.measurement.nonlinear import CartesianToBearingRange
from stonesoup.types.angle import Bearing
from stonesoup.types.detection import Detection
from stonesoup.types.groundtruth import GroundTruthState, GroundTruthPath

# Tracker Imports
from stonesoup.types.state import GaussianState

plot_type = 'static' # or 'animated'

sensor_positions = { 'RDU103': (51.52126391, 5.85862734)}

METERS_in_NM = 1852


## Get Data from BigQuery
* rdp_straight: short, straight flight path
* rdp_extended: longer flight path
* adsb_straight: truth for rdp_straight
* adsb_extended: truth for rdp_extended


In [ ]:
target_address1 = 10537421 # plane #1
target_address2 = 10610889 # plane #
# adsb_sql = f"""SELECT `timestamp`,
#         time_of_day, 
#         latitude, 
#         longitude, 
#         target_address,
#         flight_level, 
#         rho, 
#         theta
# FROM radar_data.adsb
# WHERE test_date = '2024-07-17'
# AND target_address={target_address}
# and latitude is not NULL
# AND rho<20
# ORDER BY `timestamp`"""

# adsb = dfunc.query_to_df(adsb_sql)
# data_dir = 'C:/Users/ttrinter/git_repo/Stone-Soup/data'
# adsb_file = f'{data_dir}/adsb_straight2.csv'
# adsb.to_csv(adsb_file, index=False)

# rdp_sql = f"""SELECT 
#         `timestamp`,
#         time_of_day,
#         cal, 
#         rho,
#         theta, 
#         x, 
#         y, 
#         field_note 
# FROM radar_data.rdp

# --Clutter Where Statement
# WHERE test_date = '2024-07-17'
# AND sortie_id=61
# AND `timestamp` >= '{start_time.strftime("%Y-%m-%d %H:%M:%S")}'
# AND `timestamp` <= '{end_time.strftime("%Y-%m-%d %H:%M:%S")}'
# ORDER BY `timestamp`"""

# # # Test Plane Where
# # # WHERE `timestamp` >= '{adsb_straight.timestamp.min().strftime("%Y-%m-%d %H:%M:%S")}'
# # # AND `timestamp` <= '{adsb_straight.timestamp.max().strftime("%Y-%m-%d %H:%M:%S")}'
# # # AND rho >= {adsb_straight.rho.min()-0.5}
# # # AND rho <= {adsb_straight.rho.max()+0.5}
# # # AND theta >= {adsb_straight.theta.min()- 5}
# # # AND theta <= {adsb_straight.theta.max()+5}"""

# # # # # # rdp_straight = dfunc.query_to_df(rdp_sql)
# # # # # # rdp_straight.head()

# rdp_clutter = dfunc.query_to_df(rdp_sql)
# rdp_clutter.head()

## Pass Number
All of the examples are looking at data once/second. That matches up the frequency of new plots. However, for our purposes, it may be better to look per radar pass, assuming that the radar will not see a target more than once per pass.

I should be able to set the pass number on the plots and clutter using the cat 34 sector messages.

In [ ]:
# pos_sql = """SELECT * 
#             FROM radar_data.positions 
#             WHERE sortie_id=61 
#             ORDER BY `timestamp`"""

# pos_data = dfunc.query_to_df(pos_sql)
# pos_data['timestamp'] = pos_data['timestamp'].dt.tz_localize(None)
# pos_data['timestamp'] = pos_data['timestamp'].astype('datetime64[us]')
# pos_data['timestamp'] = pos_data['timestamp'].round('ms')
# pos_data.head()

### Gather Clutter Data

In [ ]:
# # remove timezone from timestamps
# rdp_clutter['timestamp'] = rdp_clutter['timestamp'].dt.tz_localize(None)
# rdp_clutter['timestamp'] = rdp_clutter['timestamp'].astype('datetime64[us]')
# rdp_clutter['timestamp'] = rdp_clutter['timestamp'].round('ms')

# # add pass number - may use that for tracker iterations
# rdp_clutter = dfunc.add_pass_no(rdp_clutter, pos_data)

# all_matched_data['timestamp'] = all_matched_data['timestamp_rdp'].dt.tz_localize(None)
# all_matched_data['timestamp'] = all_matched_data['timestamp'].astype('datetime64[us]')
# all_matched_data['timestamp'] = all_matched_data['timestamp'].round('ms')

# # # Select only needed columns
# cols_to_keep = ['timestamp','rho','theta','cal','x','y', 'pass_no']
# rdp_clutter = rdp_clutter[cols_to_keep]

# # Left JOIN with the matched data to eliminate all RDP plots matched to any targets of opportunity
# rdp_clutter = pd.merge(rdp_clutter, all_matched_data.loc[all_matched_data.close_enough==1, ['rho_rdp','theta_rdp','timestamp']], 
#                                left_on=['rho','theta','timestamp'], 
#                                right_on = ['rho_rdp','theta_rdp','timestamp'], 
#                                how='left')

# # Drop rows that matched a target of opportunity
# rdp_clutter = rdp_clutter.loc[rdp_clutter.rho_rdp.isna()]
# # filter for only the range where the test plane is to simplify things
# rdp_clutter = rdp_clutter.loc[(rdp_clutter.rho>=matched_data.rho_rdp.min()) &
#                               (rdp_clutter.rho<=matched_data.rho_rdp.max())&
#                               (rdp_clutter.theta>=matched_data.theta_rdp.min()) &
#                               (rdp_clutter.theta<=matched_data.theta_rdp.max()) & 
#                               (rdp_clutter['timestamp'] >= matched_data.timestamp_rdp.min()) &
#                               (rdp_clutter['timestamp'] <= matched_data.timestamp_rdp.max())]

# rdp_clutter['theta_rad'] = np.deg2rad(rdp_clutter.theta)

# data_dir = 'C:/Users/ttrinter/git_repo/Stone-Soup/data'
# clutter_filename = f'{data_dir}/sample_clutter.csv'
# rdp_clutter.to_csv(clutter_filename, index=False)
# len(rdp_clutter)

## Save/Read  to/from CSV

In [ ]:
from math import pi
data_dir = 'C:/Users/ttrinter/git_repo/Stone-Soup/data'

# Now there are 2 ADSB Files
adsb_data = pd.DataFrame()
adsb_files = [f'{data_dir}/adsb_straight.csv', f'{data_dir}/adsb_straight2.csv']
for file in adsb_files:
    this_data = pd.read_csv(file)
    this_data['timestamp'] = pd.to_datetime(this_data['timestamp'], errors='coerce')
    this_data = this_data.loc[~this_data['timestamp'].isna()]
    this_data['timestamp'] = pd.to_datetime(this_data['timestamp'], errors='coerce')
    this_data['timestamp'] = this_data['timestamp'].dt.tz_localize(None)

    adsb_data = pd.concat([adsb_data, this_data])

# ADSB- first 3 min
# adsb_file = f'{data_dir}/adsb3.csv'
# adsb_data = pd.read_csv(adsb_file)

rdp_file = f'{data_dir}/rdp_straight.csv'
# rdp_straight['timestamp'] = pd.to_datetime(rdp_straight['timestamp'], errors='coerce')
# rdp_straight['theta_rad'] = np.deg2rad(rdp_straight.theta)
# rdp_straight.loc[rdp_straight.theta_rad>2*pi, 'theta_rad'] = rdp_straight.loc[rdp_straight.theta_rad>2*pi, 'theta_rad'] - 2*pi 

# rdp_straight = rdp_straight.loc[~rdp_straight['timestamp'].isna()]
# rdp_straight.to_csv(rdp_file, index=False)
# rdp_data = pd.read_csv(rdp_file)
# rdp_data['timestamp'] = pd.to_datetime(rdp_data['timestamp'], errors='coerce')
# rdp_data['timestamp'] = rdp_data['timestamp'].dt.tz_localize(None)
# rdp_data['timestamp'] = rdp_data['timestamp'].astype('datetime64[us]')
# rdp_data['timestamp'] = rdp_data['timestamp'].round('ms')


# Matched Plots
matched_csv1 = f'{data_dir}/rdp_matched.csv'
matched_csv2 = f'{data_dir}/rdp_matched2.csv'
matched_csv3min = f'{data_dir}/rdp_matched3.csv'
matched_rdp_file = matched_csv2

rdp_matched = pd.read_csv(matched_rdp_file)
rdp_matched['timestamp'] = pd.to_datetime(rdp_matched['timestamp'], errors='coerce')
rdp_matched['timestamp'] = rdp_matched['timestamp'].dt.tz_localize(None)
rdp_matched['timestamp'] = rdp_matched['timestamp'].astype('datetime64[us]')
rdp_matched['timestamp'] = rdp_matched['timestamp'].round('ms')

start_time = rdp_matched['timestamp'].min()
end_time = rdp_matched['timestamp'].max()

clutter_file = f'{data_dir}/sample_clutter.csv'
# clutter_file = f'{data_dir}/clutter3.csv'
clutter_data = pd.read_csv(clutter_file)
clutter_data['timestamp'] = pd.to_datetime(clutter_data['timestamp'], errors='coerce')
clutter_data = clutter_data.loc[~clutter_data['timestamp'].isna()]
clutter_data['timestamp'] = pd.to_datetime(clutter_data['timestamp'], errors='coerce')
clutter_data['timestamp'] = clutter_data['timestamp'].dt.tz_localize(None)

print(f'ADSB: {len(adsb_data)}')
# print(f'RDP: {len(rdp_data)}')

Make sure that there are no clutter points identical to matched points - I've already tried to remove all of these, but still see some dupes!

In [ ]:
rdp_matched['ts_10ms'] = [x.round('10ms') for x in rdp_matched['timestamp']]
clutter_data['ts_10ms'] = [x.round('10ms') for x in clutter_data['timestamp']]

In [ ]:
matched_clutter = clutter_data.merge(rdp_matched, left_on=['ts_10ms','rho','theta'], right_on=['ts_10ms','rho','theta'], how='left', suffixes=["", 'rdp_match'])
matched_clutter.loc[matched_clutter.xrdp_match.isna(), clutter_data.columns].to_csv(clutter_file, index=False)


## Shorter Files
The tracker is missing a lot of the early plots in the track for some reason. So I'll cut down the test data to the first 3 minutes to see how it does with that...

In [ ]:
# sub_start = rdp_matched.timestamp.min()
# sub_end = sub_start + timedelta(minutes=3)

# matched_rdp3 = rdp_matched.loc[(rdp_matched.timestamp>=sub_start) & (rdp_matched.timestamp<=sub_end)]
# matched_csv3min = f'{data_dir}/rdp_matched3.csv'
# matched_rdp3.to_csv(matched_csv3min, index=False)

# adsb3 = adsb_data.loc[(adsb_data.timestamp>=sub_start) & (adsb_data.timestamp<=sub_end)]
# adsb_csv3min = f'{data_dir}/adsb3.csv'
# adsb3.to_csv(adsb_csv3min, index=False)

# clutter3 = clutter_data.loc[(clutter_data.timestamp>=sub_start) & (clutter_data.timestamp<=sub_end)]
# clutter_csv3min = f'{data_dir}/clutter3.csv'
# clutter3.to_csv(clutter_csv3min, index=False)


In [ ]:
# rdp_data = pd.read_csv(rdp_file)
# rdp_data['timestamp'] = pd.to_datetime(rdp_data['timestamp'], errors='coerce')
# rdp_data['timestamp'] = rdp_data['timestamp'].dt.tz_localize(None)
# rdp_data['timestamp'] = rdp_data['timestamp'].astype('datetime64[us]')
# rdp_data['timestamp'] = rdp_data['timestamp'].round('ms')

# rdp_data = rdp_data.loc[~rdp_data['timestamp'].isna()]
# rdp_data = dfunc.add_pass_no(rdp_data, pos_data)
# rdp_data.pass_no.value_counts().sort_index()

# rdp_data.to_csv(rdp_file)

In [ ]:
# rdp_matched = pd.read_csv(matched_csv1)
# rdp_matched['timestamp'] = pd.to_datetime(rdp_matched['timestamp'], errors='coerce')
# rdp_matched['timestamp'] = rdp_matched['timestamp'].dt.tz_localize(None)
# rdp_matched['timestamp'] = rdp_matched['timestamp'].astype('datetime64[us]')
# rdp_matched['timestamp'] = rdp_matched['timestamp'].round('ms')

# rdp_matched = dfunc.add_pass_no(rdp_matched, pos_data)
# # rdp_matched.pass_no.value_counts().sort_index()

# rdp_matched.to_csv(matched_csv1, index=False)

In [ ]:
print(f'{clutter_data.timestamp.min()} : {clutter_data.timestamp.max()}')

In [ ]:
v.scatter_targets(clutter_data)
plt.suptitle('Clutter')

## Matched Data Set
To make things even simpler, I'll grab the set of matched data for this test plane. Then most of the plots should be "true" detections. Let's see how the tracker does with that.

In [ ]:
target_address1 = 10537421 # plane #1
target_address2 = 10610889 # plane #
target_addresses = [target_address1, target_address2]
file_dir = 'C:/Users/ttrinter/OneDrive - cspeed.com (1)/Documents/Data/Travis/2024-07-17'
matched_file = '20240717_Travis_matched_rdp_61.xlsx'
all_matched_data = pd.read_excel(f'{file_dir}/{matched_file}')
matched_data = all_matched_data.loc[(all_matched_data.target_address.isin(target_addresses)) &
                                (all_matched_data.close_enough==True) & 
                                (all_matched_data.timestamp_adsb>=start_time) & 
                                (all_matched_data.timestamp_adsb<=end_time)]
matched_data.head()

In [ ]:
matched_plot = v.plot_target_match2(matching=matched_data, 
                                    target_address=target_address1, 
                                    plot_show=True, 
                                    pd_loc='title')  

In [ ]:
matched_plot = v.plot_target_match2(matching=matched_data, 
                                    target_address=target_address2, 
                                    plot_show=True, 
                                    pd_loc='title')  

In [ ]:
# rdp_matched = matched_data[['timestamp_rdp',
#                             'cal_rdp',
#                             'rho_rdp',
#                             'theta_rdp', 
#                             'target_address']]

# rdp_matched['theta_rad'] = np.deg2rad(rdp_matched.theta_rdp)
# # rdp_matched.loc[rdp_matched.theta_rad>2*pi, 'theta_rad'] = rdp_straight.loc[rdp_straight.theta_rad>2*pi, 'theta_rad'] - 2*pi 
# rdp_matched.rename(columns={'rho_rdp': 'rho',
#                             'theta_rdp': 'theta', 
#                             'timestamp_rdp': 'timestamp', 
#                             'cal_rdp': 'cal'}, 
#                             inplace=True)

# rdp_matched['x'], rdp_matched['y'] = zip(*rdp_matched.apply(lambda x: dfunc.polar_to_cartesian(x.rho, x.theta), axis=1))

# matched_csv2 = f'{data_dir}/rdp_matched2.csv'
# rdp_matched.to_csv(matched_csv2, index=False)

# rdp_matched.head()

In [ ]:
# Plotting
fig, ax = plt.subplots()

# Group by the 'category' column and plot each group separately
for tgt, group in rdp_matched.groupby('target_address'):
    ax.scatter(group['x'], group['y'], label=tgt, marker="+")

# Add labels and title
plt.legend()
plt.grid()
plt.title("Matched RDP Plots")

In [ ]:
rdp_matched['x_m'] = rdp_matched.x * METERS_in_NM
rdp_matched['y_m'] = rdp_matched.y * METERS_in_NM
rdp_matched.sort_values('timestamp', inplace=True)

# Track-starts:
track_start1_t = rdp_matched.loc[rdp_matched.target_address==target_address1].iloc[0]['timestamp']
track_start1_x = rdp_matched.loc[rdp_matched.target_address==target_address1].iloc[0]['x_m']
track_start1_y = rdp_matched.loc[rdp_matched.target_address==target_address1].iloc[0]['y_m']

track_start2_t = rdp_matched.loc[rdp_matched.target_address==target_address2].iloc[0]['timestamp']
track_start2_x = rdp_matched.loc[rdp_matched.target_address==target_address2].iloc[0]['x_m']
track_start2_y = rdp_matched.loc[rdp_matched.target_address==target_address2].iloc[0]['y_m']

rdp_matched[['target_address','timestamp', 'rho','theta','x_m','y_m']].head()

In [ ]:
adsb = ADSBTruthReader.multiple_ground_truth_reader(adsb_files)
# adsb = ADSBTruthReader.single_ground_truth_reader(adsb_file)

In [ ]:
plotter = Plotter()
plotter.plot_ground_truths(adsb, 
                        mapping=[0, 2], 
                        markersize = 5,
                        marker = 's', 
                        markerfacecolor = 'none', 
                        alpha = 0.2)
plt.grid()
plt.title("Two Targets of Opportunity")

In [ ]:
# Detections
matched_xy = CSVReaderXY(matched_rdp_file)
matched_polar = CSVReaderPolar(matched_rdp_file)

# Cutter
clutter = CSVClutterReaderXY(clutter_file)

dets = [next(iter(detection[1])) for detection in matched_xy.detections_gen()]
cluts = [next(iter(detection[1])) for detection in clutter.detections_gen()]

# Combine detections with clutter
all_measurements = dets + cluts
all_measurements.sort(key=lambda obj: obj.timestamp)

plot_all(start_time, 
         end_time,
         all_measurements=all_measurements, 
         adsb=adsb, 
         plot_type='static')

## From Tutorial #6

In [ ]:
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, \
                                               ConstantVelocity
from stonesoup.predictor.kalman import KalmanPredictor
from stonesoup.updater.kalman import KalmanUpdater
from stonesoup.predictor.kalman import UnscentedKalmanPredictor
from stonesoup.updater.kalman import UnscentedKalmanUpdater

from stonesoup.types.track import Track
from stonesoup.hypothesiser.distance import DistanceHypothesiser
from stonesoup.measures import Mahalanobis, Euclidean

from stonesoup.dataassociator.neighbour import NearestNeighbour

In [ ]:
# Measurement Model
np.random.seed(42)
default_variance = 50

measurement_model = LinearGaussian(
    ndim_state=4,   # Number of state dimensions (position and velocity in 2D)
    mapping=(0, 2), # Mapping measurement vector index to state index
    noise_covar=np.array([[default_variance, 0 ],  
                          [0, default_variance]])
    )  #Covariance matrix for Gaussian PDF

# Transition Model
q_const = 60
q_x = q_const
q_y = q_const
transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(q_x),
                                                          ConstantVelocity(q_y)])

In [ ]:
predictor = KalmanPredictor(transition_model)
updater = KalmanUpdater(measurement_model)

# predictor = UnscentedKalmanPredictor(transition_model)
# updater = UnscentedKalmanUpdater(measurement_model)  # Keep alpha as default = 0.5

hypothesiser = DistanceHypothesiser(predictor, updater, measure=Mahalanobis(), missed_distance=3)
# hypothesiser = DistanceHypothesiser(predictor, updater, measure=Euclidean(), missed_distance=80)
data_associator = NearestNeighbour(hypothesiser)

# Clear things out from prior runs
hypothesis = None
post = None

prior1 = GaussianState([[track_start1_x], [1], [track_start1_y], [1]], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)
prior2 = GaussianState([[track_start2_x], [1], [track_start2_y], [1]], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)

# create a prior using the location of the radar
# prior = GaussianState([[0], [q_const], [0], [q_const]], np.diag([default_variance, 0.5, default_variance, 0.5]), timestamp=start_time)

# Loop through the predict, hypothesise, associate and update steps.
# track = Track([prior1])
if "tracks" in globals():
    del tracks

if "track" in globals():
    del(track)

tracks = {Track([prior1]), Track([prior2])}

grouped_sec, grouped_pass = group_plots(all_measurements)

# for n, measurements in enumerate(grouped_sec):
for n, measurements in enumerate(grouped_pass):    
    this_time = min(measurements, key=lambda meas: meas.timestamp).timestamp
    this_time = this_time.replace(microsecond=0)

    # Calculate all hypothesis pairs and associate the elements in the best subset to the tracks.
    if len(measurements)>0:
    # for n, measurements in enumerate(dets):
        try: 
            hypotheses = data_associator.associate(tracks,
                                                measurements,
                                                this_time)
            for track in tracks:
                hypothesis = hypotheses[track]
                if hypothesis.measurement:
                    post = updater.update(hypothesis)
                    track.append(post)
                else:  # When data associator says no detections are good enough, we'll keep the prediction
                    track.append(hypothesis.prediction)

        except:
            # print(f'{this_time}: {len(measurements)}: ERROR')
            continue


## Plot the ground truth and measurements with clutter.

In [ ]:
plot_all(start_time,
         end_time,
         all_measurements=all_measurements, 
         adsb=adsb,
         tracks=tracks,
         plot_type='static')
        # plot_type='animated')

This tracker did a good job of following each of the tracks separately! However, the more vertical track was wierdly combined with what appears to be an entirely different track in another location and direction.

Similar to the previous analysis - after running the polar tracks, re-running these cartesian tracks no longer works! Clearly something is lingering after a run. Not all inputs are getting reset with a re-run. Need to sort that out!

It is also suspect that 14 correlated plots get passed and are not added to the first track. It is not clear what/why these plots are skipped. Need to investigate that too. Perhaps with a smaller dataset that only inlcudes the first 20 correlated plots or something.

Maybe the subsequent tutorials will address track deletions and initiation.


## Polar Coordinates
Trying again, but changing the process to read rho and theta and, maybe later also radial velocity.

In [ ]:
# Detections
meas_polar = CSVReaderPolar(matched_csv2)

dets = [next(iter(detection[1])) for detection in meas_polar.detections_gen()]
cluts = [next(iter(detection[1])) for detection in clutter.detections_gen()]

# Combine detections with clutter
all_measurements = dets + cluts
all_measurements.sort(key=lambda obj: obj.timestamp)


### Kalman Filtering Again...
Should be the same from here forward.

In [ ]:
# Transition Model
q_const = 35
q_x = q_const
q_y = q_const
default_variance=50

transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(q_x),
                                                          ConstantVelocity(q_y)])

# measurement_model = CartesianToBearingRange(ndim_state=4, 
#                                             mapping=(0,2), 
#                                             noise_covar=np.array([[default_variance, 0 ],
#                                                                   [0, default_variance]]))

# Create prior
# predictor = KalmanPredictor(transition_model)
# updater = KalmanUpdater(measurement_model)

predictor = UnscentedKalmanPredictor(transition_model)
updater = UnscentedKalmanUpdater(measurement_model)  # Keep alpha as default = 0.5

hypothesiser = DistanceHypothesiser(predictor, updater, measure=Mahalanobis(), missed_distance=3)
data_associator = NearestNeighbour(hypothesiser)

# create a prior using the approximate start of the track
# prior = GaussianState([[track_start_x], [1], [track_start_y], [1]], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)

prior1 = GaussianState([[track_start1_x], [1], [track_start1_y], [1]], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)
prior2 = GaussianState([[track_start2_x], [1], [track_start2_y], [1]], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)

# create a prior using the location of the radar
# prior = GaussianState([[0], [q_const], [0], [q_const]], np.diag([default_variance, 0.5, default_variance, 0.5]), timestamp=start_time)

# Loop through the predict, hypothesise, associate and update steps.
# del(tracks)
if "tracks" in globals():
    del tracks

if "track" in globals():
    del(track)

tracks = {Track([prior1]), Track([prior2])}

grouped_sec, grouped_pass = group_plots(all_measurements)

# for n, measurements in enumerate(grouped_sec):
for n, measurements in enumerate(grouped_pass):    
    this_time = min(measurements, key=lambda meas: meas.timestamp).timestamp
    this_time = this_time.replace(microsecond=0)

    # Calculate all hypothesis pairs and associate the elements in the best subset to the tracks.
    if len(measurements)>0:
    # for n, measurements in enumerate(dets):
        try: 
            hypotheses = data_associator.associate(tracks,
                                                measurements,
                                                this_time)
            for track in tracks:
                hypothesis = hypotheses[track]
                if hypothesis.measurement:
                    post = updater.update(hypothesis)
                    track.append(post)
                else:  # When data associator says no detections are good enough, we'll keep the prediction
                    track.append(hypothesis.prediction)

        except:
            # print(f'{this_time}: {len(measurements)}: ERROR')
            continue


In [ ]:
plot_all(start_time,
         end_time,
         all_measurements=all_measurements, 
         adsb=adsb,
         tracks=tracks,
         plot_type='static')

It is unclear why the cartesian tracker worked reasonably well (besides continuing the track with a large change in direction and location) and the polar tracker didn't ever continue the track.

Leaving it for the next tutorial.

In [ ]:
len(grouped_sec)